<a href="https://colab.research.google.com/github/financieras/campus/blob/main/pilas/pilas33.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algotirno Moder Times llevando a Excel dos opciones
* En caso de empate en el mínimo total:
 - una opción elige el primer mínimo
 - la otra opción elige el último mínimo

In [ ]:
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import openpyxl                # importamos la librería openpyxl

wb = openpyxl.Workbook()       # para crear por primera vez un libro
ws = wb.active                 # estas dos lineas crean el libro, aún sin nombre

wb.save('dos_vias.xlsx')          # grabamos el fichero por primera vez

In [ ]:
# FUNCIONES
def sa(a, b, result):   # result es un array que va acumulando los pasos dados. Ejemplo: result = ['pb','ra','sa','rra']
    if len(a) > 1: a[0],a[1] = a[1],a[0]; result.append('sa')
    return a, b, result
def sb(a, b, result):
    if len(b) > 1: b[0],b[1] = b[1],b[0]; result.append('sb')
    return a, b, result
def ss(a, b, result):   # ss no llama a sa y sb ya que en result quedarían anotadas las tres funciones: ss, sa y sb
    if len(a) > 1 or len(b) > 1:
        result.append('ss')
        if len(a) > 1: a[0],a[1] = a[1],a[0]
        if len(b) > 1: b[0],b[1] = b[1],b[0]
    return a, b, result
def pa(a, b, result):
    if len(b) > 0:
        a.insert(0, b[0])
        b.pop(0); result.append('pa')
    return a, b, result
def pb(a, b, result):
    if len(a) > 0:
        b.insert(0, a[0])
        a.pop(0); result.append('pb')
    return a, b, result
def ra(a, b, result):
    if len(a) > 1: a.append(a.pop(0)); result.append('ra')
    return a, b, result
def rb(a, b, result):
    if len(b) > 1: b.append(b.pop(0)); result.append('rb')
    return a, b, result
def rr(a, b, result):
    if len(a) > 1 or len(b) > 1:
        result.append('rr')
    if len(a) > 1: a.append(a.pop(0))
    if len(b) > 1: b.append(b.pop(0))
    return a, b, result
def rra(a, b, result):
    if len(a) > 1: a.insert(0, a.pop()); result.append('rra')
    return a, b, result
def rrb(a, b, result):
    if len(b) > 1: b.insert(0, b.pop()); result.append('rrb')
    return a, b, result
def rrr(a, b, result):
    if len(a) > 1 or len(b) > 1:
        result.append('rrr')
    if len(a) > 1: a.insert(0, a.pop())
    if len(b) > 1: b.insert(0, b.pop())
    return a, b, result

In [ ]:
# PASOS NECESARIOS PARA COLOCAR CADA ELEMENTO DE A EN SU SITIO EN B      total = pasosA + pasosB (esta suma es solo una idea, se ha de sumar de una forma peculiar)

def necesariosA(a, b):   # array pasosA calcula los pasos necesarios para colocar cada elemento de A como el primero de su pila
    for v in a:
        if a.index(v) < len(a)/2:
            pasosA.append(a.index(v))
        else:
            pasosA.append(-(len(a)- a.index(v)))

def necesariosB(a, b):   # array pasosB calcula los pasos de B necesarios para colocar cada elemento de A dentro de su sitio en B
    for i in range(len(a)):    # objetivo_primero es el número que se ha de poner en la primera posición de la pila B
        if a[i] < min(b):      # si el elemento de A considerado es menor que el mayor de B entonces
            objetivo_primero = max(b)   # el objetivo_primero será el mayor de B
        else:   # objetivo_primero en este caso será el maximo de los inferiores en B al valor iésimo de A
            objetivo_primero = min(b)   #se inicializa en el valor mínimo de la pila B
            for j in range(len(b)):
                if b[j] < a[i] and b[j] > objetivo_primero:
                        objetivo_primero = b[j]
        # el objetivo_primero se ha de situar el primero de la pila B            
        if b.index(objetivo_primero) < len(b)/2:
            pasosB.append(b.index(objetivo_primero))
        else:
            pasosB.append(-(len(b)- b.index(objetivo_primero)))

def totaliza(a, b):   # totalizar pasos
    global total
    for i in range(len(pasosA)):
        if pasosA[i] * pasosB[i] < 0:   # si son de distinto signo, uno positivo y otro negativo
            total.append(abs(pasosA[i]) + abs(pasosB[i]))   # no hay sinergia
        else:   # si son de igual signo o alguno cero
            total.append(max(abs(pasosA[i]), abs(pasosB[i])))  # si son de igual signo hay sinergia

def calculaIndexPasosMinimos(opcion):   # retorna el índice del elemento de la pila A que menos pasos totales necesita
    global pasosA, pasosB, total
    pasosA = []   # [0,1,2, ...,41,-41,... , -2,-1]  vector donde cada index está asociado con el valor del mismo index en A
    pasosB = []   # [-3,2,-5, ..., -5,0]   estos dos arrays se han de recalcular cada vez que realmente se mueva algún elemento de A a B
    total = []
    necesariosA(a, b)
    necesariosB(a, b)
    totaliza(a, b)
    all_min = [i for i,v in enumerate(total) if v == min(total)]   # da un array con todos los index del valor mínimo
    return all_min[-opcion]   # si se alcanza el objetivo probar con all_min[-1]

In [ ]:
def giraA(a, b, indice, steps, result):
    for i in range(abs(steps)):
        if steps > 0:
            ra(a, b, result)
        elif steps < 0:
            rra(a, b, result)

def giraB(a, b, indice, steps, result):
    for i in range(abs(steps)):
        if steps > 0:
            rb(a, b, result)
        elif steps < 0:
            rrb(a, b, result)

# girando pilas A y B
def giraPilas(a, b, indice, result):   # indice es el index del valor en la pila A que deseamos poner el primero
    if pasosA[indice] * pasosB[indice] > 0:   # Existe sinergia, nos podemos ahorrar pasos
        pasos_comunes = min(abs(pasosA[indice]), abs(pasosB[indice]))
        for i in range(pasos_comunes):
            if pasosA[indice] > 0:   # si el signo de ambos es positivo, ya que ambos tienen el mismo signo
                rr(a, b, result)
            elif pasosA[indice] < 0: # si el signo de ambos es negativo
                rrr(a, b, result)
        exceso_pasosA = abs(pasosA[indice]) - pasos_comunes
        exceso_pasosB = abs(pasosB[indice]) - pasos_comunes
        giraA(a, b, indice, ((pasosA[indice] > 0) - (pasosA[indice] < 0)) * exceso_pasosA, result)    # (a > 0) - (a < 0) da el signo de a
        giraB(a, b, indice, ((pasosB[indice] > 0) - (pasosB[indice] < 0)) * exceso_pasosB, result)   # Python no tiene función sign
    else:   # No existe sinergia
        giraA(a ,b, indice, pasosA[indice], result)   # gira A
        giraB(a, b, indice, pasosB[indice], result)   # gira B

In [ ]:
def situarMax_en_B(a, b, result):   # situa el valor máximo de B en la primera posición de B
    indice = b.index(max(b))
    if indice < len(a)/2:
        steps = indice
    else:
        steps = -(len(b)- indice)
    giraB(a, b, indice, steps, result)

def crecientesA(a, b, result):   # situa los tres valores de A en forma creciente estricta: Ejemplo: 17, 45, 82
    if a[0] < a[1] > a[2] and a[0] < a[2]:        # caso: 1 3 2
        ra(a, b, result)
        sa(a, b, result)
        rra(a, b, result)
    elif a[0] > a[1] < a[2] and a[0] < a[2]:        # caso: 2 1 3
        sa(a, b, result)
    elif a[0] < a[1] > a[2] and a[0] > a[2]:        # caso: 2 3 1
        rra(a, b, result)
    elif a[0] > a[1] < a[2] and a[0] > a[2]:        # caso: 3 1 2
        ra(a, b, result)
    elif a[0] > a[1] > a[2]:                        # caso: 3 2 1
        sa(a, b, result)
        rra(a, b, result)
    return                                          # caso: 1 2 3 en este caso no se hace nada pq ya están ordenados

In [ ]:
# Después de ordenar los tres elementos de A en orden creciente estricto con la función crecientesA

def cremallera(a, b, result):   # función que va pasando los números de B a A en forma de cremallera, intercalándolos con los que ya hay en A
    aux = [a[-3], a[-2], a[-1]] # creamos un array aux con los tres últimos valores de A
    while len(aux) > 0:         # Mientras aux tenga elementos
        maximo = max(b + aux)   # calcula el maximo entre b y aux
        if maximo in b:         # si el maximo está en B hacer pa
            pa(a, b, result)
        if maximo in aux:       # si el máximo está en aux hacer rra y aux.pop
            rra(a, b, result)
            aux.pop()
    while len(b) > 0:           # ahora aux está vacío y solo queda hacer pa todo el rato
        pa(a, b, result)

In [ ]:
if __name__ == "__main__":
    from random import seed, shuffle
    seed()
    n = 500                             # número de elementos de la pila
    for muestra in range(100):
        a = list(range(1, n+1)); shuffle(a) # generación aleatoria de la pila A
        b = []
        a_original = a[:]
        ws = wb["Sheet"]                    # accediendo a la hoja Sheet
        ws.cell(row=muestra+11, column=2).value = muestra + 1
        ws.cell(row=muestra+11, column=3).value = str(a_original)
        for opcion in range(2):            # vamos a probar a elegir en caso de empate en el total el primero o el último
            a = a_original[:]
            b = []
            result = []                         # recoje los pasos dados. Ejemplo: result = ['pb','ra','sa','rra']
            pb(a, b, result); pb(a, b, result)  # pasa de A a B la pareja de elementos de A elegidos a[0], a[1]
            while len(a) > 3:
                index_minimosPasos = calculaIndexPasosMinimos(opcion)
                giraPilas(a, b, index_minimosPasos, result)
                pb(a, b, result)         # lo pasa de A a B haciendo pb
            situarMax_en_B(a, b, result) # situa el valor máximo de B en la primera posición de B
            crecientesA(a, b, result)    # situa los tres valores de A en forma creciente estricta: Ejemplo: 17, 45, 82
            cremallera(a, b, result)     # función que va pasando los números de B a A en forma de cremallera, intercalándolos con los que ya hay en A
            nresult = len(result)
            ws.cell(row=muestra+11, column=opcion+4).value = nresult
        wb.save('dos_vias.xlsx')
    wb.close()